# 드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd '/content/drive/MyDrive/Colab Notebooks'

/content/drive/MyDrive/Colab Notebooks


# 이미지 전처리

1. 필요한 라이브러리 import

*   PIL : 이미지 전처리
*   glob: 디렉토리에서 이미지 데이터 불러오기
*   numpy: 이미지 벡터화
*   sklearn: 모델 학습

In [ ]:
from PIL import Image
import os, glob
import numpy as np
from sklearn.model_selection import train_test_split

2. 분류 대상 카테고리(라벨) 선택
*   라벨

```
categories=["top","pants"]
```

In [ ]:
# 분류 대상 카테고리 선택
caltech_dir='/content/drive/MyDrive/Colab Notebooks/data2'
categories=["top","pants"]
nb_classes=len(categories)


3. 데이터 전처리
*   이미지 데이터 벡터화
*   train/test set 분리-->7:3 비율로



In [ ]:
# 이미지 크기 지정
image_w=256
image_h=256
pixels=image_w*image_h*3

# 이미지 데이터 읽기
X=[]
Y=[]
for idx, cat in  enumerate(categories):
  # 레이블 지정
  label=[0 for i in range(nb_classes)]
  label[idx]=1
  
  image_dir= caltech_dir+ '/' + str(cat)
  files=glob.glob(image_dir+"/*.jpg")
  for i, f in enumerate(files):
    img=Image.open(f)
    img=img.convert("RGB")
    img=img.resize((image_w,image_h))
    data=np.asarray(img)
    X.append(data)
    Y.append(label)
    # if i%10==0:
      # print(i,"\n",data)
X=np.array(X)
Y=np.array(Y)

# train/ test split
X_train,X_test,y_train, y_test=train_test_split(X,Y,test_size=0.30)
xy=(X_train,X_test,y_train,y_test)
np.save("/content/drive/MyDrive/Colab Notebooks/data2/img_obj.npy",xy)
print("ok,",len(Y))



/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


ok, 1668


# 모델 학습
1. 필요한 라이브러리, 데이터셋 불러오기

In [ ]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
import numpy as np

# 카테고리 지정하기
categories = ["top","pants"]
nb_classes = len(categories)
# 이미지 크기 지정하기
image_w = 256 
image_h = 256

# 데이터 불러오기 --- (※1)
X_train, X_test, y_train, y_test = np.load("/content/drive/MyDrive/Colab Notebooks/data2/img_obj.npy",allow_pickle=True)
# 데이터 정규화하기
X_train = X_train.astype("float") / 256
X_test  = X_test.astype("float")  / 256
print('X_train shape:', X_train.shape)

X_train shape: (1167, 256, 256, 3)


2. 모델 설계

*   keras를 이용하여 CNN 구조 설계
*   optimizer: adam
*   loss function: cross entropy
*   activate function: relu/softmax

In [ ]:
# 모델 구축하기
model = Sequential()
model.add(Convolution2D(32, 3, 3, padding='same',input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64, 3, 3, padding='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(MaxPooling2D(pool_size=(1, 1)))
model.add(Dropout(0.25))

model.add(Convolution2D(64, 3, 3, padding='same'))
model.add(Activation('relu'))
# model.add(Convolution2D(64, 3, 3))
model.add(MaxPooling2D(pool_size=(1, 1)))
model.add(Dropout(0.25))

model.add(Flatten()) # --- (※3) 
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

3. 모델 훈련 및 평가
*   epoch: 35
*   batch: 32
*   dropout: 0.25

In [ ]:
# 모델 훈련하기 --- (※4)
model.fit(X_train, y_train, batch_size=32, epochs=50)
    
# 모델 평가하기--- (※5)
score = model.evaluate(X_test, y_test)
print('loss=', score[0])
print('accuracy=', score[1])

Epoch 1/50
37/37 [==============================] - 2s 21ms/step - loss: 0.6377 - accuracy: 0.6227
Epoch 2/50
37/37 [==============================] - 1s 18ms/step - loss: 0.4436 - accuracy: 0.7937
Epoch 3/50
37/37 [==============================] - 1s 18ms/step - loss: 0.3658 - accuracy: 0.8265
Epoch 4/50
37/37 [==============================] - 1s 18ms/step - loss: 0.2421 - accuracy: 0.8982
Epoch 5/50
37/37 [==============================] - 1s 18ms/step - loss: 0.2205 - accuracy: 0.9065
Epoch 6/50
37/37 [==============================] - 1s 18ms/step - loss: 0.1392 - accuracy: 0.9369
Epoch 7/50
37/37 [==============================] - 1s 17ms/step - loss: 0.1302 - accuracy: 0.9429
Epoch 8/50
37/37 [==============================] - 1s 18ms/step - loss: 0.1448 - accuracy: 0.9409
Epoch 9/50
37/37 [==============================] - 1s 18ms/step - loss: 0.1427 - accuracy: 0.9514
Epoch 10/50
37/37 [==============================] - 1s 18ms/step - loss: 0.0926 - accuracy: 0.9681
Epoch 11/

4. 모델 저장

In [ ]:
model.save("binary_class_cloth_more.h5")